In [5]:
from pyomo.environ import *
from pyomo.opt import *
opt = solvers.SolverFactory("glpk")

model = ConcreteModel()

model.x = Var([1,2], within=NonNegativeReals)

model.c1 = Constraint(expr = 6* model.x[1] + 4* model.x[2] <= 40)
model.c2 = Constraint(expr = 8*model.x[1] + 4* model.x[2] <= 40)
model.c3 = Constraint(expr = 3*model.x[1] + 3*model.x[2] <= 20)
model.z = Objective(expr = 300*model.x[1] + 200*model.x[2], sense=maximize)
# add dual
model.dual = Suffix(direction=Suffix.IMPORT)
results = opt.solve(model)

print(f"model: {model.x.get_values()}" )

model: {1: 3.33333333333333, 2: 3.33333333333333}


In [6]:
#now we can compute the dual values
i =1
for m in [model.c1, model.c2, model.c3]:
    print(f"model: c{i}, dual: {model.dual[m]}")
    i+=1


model: c1, dual: 0.0
model: c2, dual: 25.0
model: c3, dual: 33.3333333333333


Occasionally, someone stops by asking for help with restoring antique clocks. How
much should David charge per hour for mechanical repairs and how much should
LaDeana charge per hour for wood restoration assuming that they do not wish to
add more working hours and also do not wish to reduce company profit if one of them
is taking on a repair job?

David does not need to charge anything per hour for mechanical repairs since he is not the bottleneck (obviously he should charge something), but company profit is not reduced if he takes a repair job as long as the amount of time taken for repair jobs is not too large

LaDeana should charge at least $ 25.000 / hr

Problem 2:

In [9]:
#following Petrat

model2 = ConcreteModel()

A = [1,2,3]# destinations
R = ['D1', 'D2', 'D3' , 'D4'] # distribution centers
w={ (1, 'D1'): 800, (1, 'D2'): 1300, (1, 'D3'): 400, (1, 'D4'): 700,
    (2, 'D1'): 1100,(2, 'D2'): 1400, (2, 'D3'): 600, (2, 'D4'): 1000,
    (3, 'D1'): 600, (3, 'D2'): 1200, (3, 'D3'): 800, (3, 'D4'): 900
}# constraint matrix
s = { 1:12, 2: 17, 3: 11} #supply constraints
d = {'D1':10,'D2':10,'D3':10,'D4':10} # demand constraints
def supply_rule(model, i):
    return sum(model.x[i,j] for j in R) == s[i]

def demand_rule(model, j):
    return sum(model.x[i,j] for i in A) == d[j]


model2.x = Var(A,R, within=NonNegativeReals)
# I don't think this expression correctly models what is meant by the objective fct, oh well
# do you homogeneity for this to work?
model2.z = Objective(expr=sum( (    100 + 0.50*w[i,j]) * model2.x[i,j] for j in R for i in A), sense=minimize)
#set constraints
model2.supply = Constraint(A, rule=supply_rule)
model2.demand=Constraint(R, rule=demand_rule)



model2.dual = Suffix(direction=Suffix.IMPORT)
results = opt.solve(model2)

print(model2.x.get_values())
print(f"total cost of transportation: {model2.z.expr()}")


{(1, 'D1'): 0.0, (1, 'D2'): 0.0, (1, 'D3'): 2.0, (1, 'D4'): 10.0, (2, 'D1'): 0.0, (2, 'D2'): 9.0, (2, 'D3'): 8.0, (2, 'D4'): 0.0, (3, 'D1'): 10.0, (3, 'D2'): 1.0, (3, 'D3'): 0.0, (3, 'D4'): 0.0}
total cost of transportation: 20200.0


Now suppose that demand in the area served by Center 1 goes up to 15 shipments
per month. Production cannot be increased on short notice, so some or all of the
distribution centers will be under-supplied. Modify your Pyomo code to determine
the total number of shipments to arrive at each of the centers if the objective is still
to minimize the overall cost of transportation.

In [12]:
#copy paste because I am lazy
model3 = ConcreteModel()

A = [1,2,3]# destinations
R = ['D1', 'D2', 'D3' , 'D4'] # distribution centers
w={ (1, 'D1'): 800, (1, 'D2'): 1300, (1, 'D3'): 400, (1, 'D4'): 700,
    (2, 'D1'): 1100,(2, 'D2'): 1400, (2, 'D3'): 600, (2, 'D4'): 1000,
    (3, 'D1'): 600, (3, 'D2'): 1200, (3, 'D3'): 800, (3, 'D4'): 900
}# constraint matrix
s = { 1:12, 2: 17, 3: 11} #supply constraints
d = {'D1':15,'D2':10,'D3':10, 'D4':10} # demand constraints, modify first one
def supply_rule_mod(model3, i):
    #demand still has to be met
    return sum(model3.x[i,j] for j in R) == s[i] # all supply must be completed

def demand_rule_mod(model3, j):
    #add a less than equal sign here b/c we can't fulfill all orders now
    return sum(model3.x[i,j] for i in A) <= d[j]


model3.x = Var(A,R, within=NonNegativeReals)

model3.z = Objective(expr=sum( (100 + 0.50*w[i,j] )* model3.x[i,j] for j in R for i in A), sense=minimize)
#set constraints
model3.supply = Constraint(A, rule=supply_rule_mod)
model3.demand=Constraint(R, rule=demand_rule_mod)



model3.dual = Suffix(direction=Suffix.IMPORT)
results = opt.solve(model3)

print(model3.x.get_values())
print(f"total cost of transportation: {model3.z.expr()}")


{(1, 'D1'): 0.0, (1, 'D2'): 0.0, (1, 'D3'): 2.0, (1, 'D4'): 10.0, (2, 'D1'): 0.0, (2, 'D2'): 9.0, (2, 'D3'): 8.0, (2, 'D4'): 0.0, (3, 'D1'): 10.0, (3, 'D2'): 1.0, (3, 'D3'): 0.0, (3, 'D4'): 0.0}
total cost of transportation: 20200.0


fixed ? 